In [29]:
# Imports the Google Cloud client library
from google.cloud import storage

# Instantiates a client
storage_client = storage.Client()

# The name for the new bucket
bucket_name = "budfox-data-science-development"

# Creates the new bucket
bucket = storage_client.get_bucket(bucket_name)

print("Bucket {} created.".format(bucket.name))
# Note: Client.list_blobs requires at least package version 1.17.0.
blobs = storage_client.list_blobs(bucket_name)

assets = [blob.name.split("/")[-1].split(".")[0] for blob in blobs if blob.name.split("/")[-2].split(".")[0]=='Data']
print(len(assets))
uris = ["gs://budfox-data-science-development/Data/" + asset + ".csv" for asset in assets]
for i, uri in enumerate(uris):
    if i > 10:
        break
    else:
        print(i, uri)

Bucket budfox-data-science-development created.
251
0 gs://budfox-data-science-development/Data/AAVEETH.csv
1 gs://budfox-data-science-development/Data/AAVEEUR.csv
2 gs://budfox-data-science-development/Data/AAVEGBP.csv
3 gs://budfox-data-science-development/Data/AAVEUSD.csv
4 gs://budfox-data-science-development/Data/AAVEXBT.csv
5 gs://budfox-data-science-development/Data/ADAETH.csv
6 gs://budfox-data-science-development/Data/ADAEUR.csv
7 gs://budfox-data-science-development/Data/ADAUSD.csv
8 gs://budfox-data-science-development/Data/ADAUSDT.csv
9 gs://budfox-data-science-development/Data/ADAXBT.csv
10 gs://budfox-data-science-development/Data/ALGOETH.csv


In [32]:
blobs = storage_client.list_blobs(bucket_name)
print(assets)

['AAVEETH', 'AAVEEUR', 'AAVEGBP', 'AAVEUSD', 'AAVEXBT', 'ADAETH', 'ADAEUR', 'ADAUSD', 'ADAUSDT', 'ADAXBT', 'ALGOETH', 'ALGOEUR', 'ALGOUSD', 'ALGOXBT', 'ANTETH', 'ANTEUR', 'ANTUSD', 'ANTXBT', 'ATOMETH', 'ATOMEUR', 'ATOMUSD', 'ATOMXBT', 'AUDJPY', 'AUDUSD', 'BALETH', 'BALEUR', 'BALUSD', 'BALXBT', 'BATETH', 'BATEUR', 'BATUSD', 'BATXBT', 'BCHAUD', 'BCHETH', 'BCHEUR', 'BCHGBP', 'BCHJPY', 'BCHUSD', 'BCHUSDT', 'BCHXBT', 'COMPETH', 'COMPEUR', 'COMPUSD', 'COMPXBT', 'CRVETH', 'CRVEUR', 'CRVUSD', 'CRVXBT', 'DAIEUR', 'DAIUSD', 'DAIUSDT', 'DASHEUR', 'DASHUSD', 'DASHXBT', 'DOTETH', 'DOTEUR', 'DOTUSD', 'DOTUSDT', 'DOTXBT', 'EOSETH', 'EOSEUR', 'EOSUSD', 'EOSUSDT', 'EOSXBT', 'ETCETH', 'ETCEUR', 'ETCUSD', 'ETCXBT', 'ETHAUD', 'ETHCAD', 'ETHCHF', 'ETHDAI', 'ETHEUR', 'ETHGBP', 'ETHJPY', 'ETHUSD', 'ETHUSDC', 'ETHUSDT', 'ETHXBT', 'EURAUD', 'EURCAD', 'EURCHF', 'EURGBP', 'EURJPY', 'EURUSD', 'FILETH', 'FILEUR', 'FILUSD', 'FILXBT', 'GBPUSD', 'GNOETH', 'GNOEUR', 'GNOUSD', 'GNOXBT', 'GRTETH', 'GRTEUR', 'GRTUSD', 'G

In [34]:
from google.cloud import bigquery

# Construct a BigQuery client object.
client = bigquery.Client()

In [13]:
# TODO(developer): Set table_id to the ID of the table to create.
# table_id = "your-project.your_dataset.your_table_name"

job_config = bigquery.LoadJobConfig(
    schema=[
        bigquery.SchemaField("time_stamp", "TIMESTAMP"),
        bigquery.SchemaField("price", "FLOAT"),
        bigquery.SchemaField("volume", "FLOAT")#, 
        #bigquery.SchemaField("asset", "STRING")
    ],
    skip_leading_rows=0,
    # The source format defaults to CSV, so the line below is optional.
    source_format=bigquery.SourceFormat.CSV,
)
for uri, table in zip(uris, assets):
        
    table_id= "development-302415.markets." + table 
    load_job = client.load_table_from_uri(
        uri, table_id, job_config=job_config
    )  # Make an API request.

    load_job.result()  # Waits for the job to complete.

    destination_table = client.get_table(table_id)  # Make an API request.
    print("Loaded {} rows.".format(destination_table.num_rows))

Loaded 1461 rows.
Loaded 6248 rows.
Loaded 368 rows.
Loaded 6088 rows.
Loaded 2379 rows.
Loaded 225581 rows.
Loaded 1612232 rows.
Loaded 1178759 rows.
Loaded 15086 rows.
Loaded 798295 rows.
Loaded 51289 rows.
Loaded 258108 rows.
Loaded 244308 rows.
Loaded 134059 rows.
Loaded 2043 rows.
Loaded 9860 rows.
Loaded 7720 rows.
Loaded 2816 rows.
Loaded 117767 rows.
Loaded 511124 rows.
Loaded 414077 rows.
Loaded 374009 rows.
Loaded 1763 rows.
Loaded 21700 rows.
Loaded 4266 rows.
Loaded 21953 rows.
Loaded 17113 rows.
Loaded 9991 rows.
Loaded 38792 rows.
Loaded 238358 rows.
Loaded 188388 rows.
Loaded 146902 rows.
Loaded 4035 rows.
Loaded 26663 rows.
Loaded 5279798 rows.
Loaded 7509 rows.
Loaded 629 rows.
Loaded 4838190 rows.
Loaded 52831 rows.
Loaded 2964327 rows.
Loaded 45758 rows.
Loaded 117022 rows.
Loaded 119265 rows.
Loaded 66429 rows.
Loaded 11498 rows.
Loaded 109955 rows.
Loaded 87971 rows.
Loaded 32822 rows.
Loaded 206931 rows.
Loaded 244445 rows.
Loaded 96450 rows.
Loaded 1628570 rows.


In [35]:
queries = ["SELECT time_stamp, price, volume, '" + asset + "' as asset FROM `development-302415.markets." + asset + "`" for asset in assets]
for q in queries[:3]:
    print(q)

SELECT time_stamp, price, volume, 'AAVEETH' as asset FROM `development-302415.markets.AAVEETH`
SELECT time_stamp, price, volume, 'AAVEEUR' as asset FROM `development-302415.markets.AAVEEUR`
SELECT time_stamp, price, volume, 'AAVEGBP' as asset FROM `development-302415.markets.AAVEGBP`


In [30]:
%%bigquery results
SELECT time_stamp, price, volume, 'AAVEEUR' as asset FROM development-302415.markets.AAVEEUR

In [31]:
results

,time_stamp,price,volume,asset
0,2020-12-23 22:21:10+00:00,56.00,1.497604,AAVEEUR
1,2020-12-23 22:21:10+00:00,56.00,0.002392,AAVEEUR
2,2020-12-23 22:21:10+00:00,56.00,0.000004,AAVEEUR
3,2020-12-23 22:21:10+00:00,56.00,7.310005,AAVEEUR
4,2020-12-23 22:21:10+00:00,56.00,0.011677,AAVEEUR
...,...,...,...,...
6243,2020-12-18 13:33:17+00:00,69.84,0.100100,AAVEEUR
6244,2020-12-22 02:07:44+00:00,66.59,0.100100,AAVEEUR
6245,2020-12-21 00:37:03+00:00,70.59,0.100100,AAVEEUR
6246,2020-12-18 22:24:36+00:00,71.09,0.100100,AAVEEUR


In [36]:
giant_query = " UNION ALL ".join(queries)

In [37]:
print(giant_query)

SELECT time_stamp, price, volume, 'AAVEETH' as asset FROM `development-302415.markets.AAVEETH` UNION ALL SELECT time_stamp, price, volume, 'AAVEEUR' as asset FROM `development-302415.markets.AAVEEUR` UNION ALL SELECT time_stamp, price, volume, 'AAVEGBP' as asset FROM `development-302415.markets.AAVEGBP` UNION ALL SELECT time_stamp, price, volume, 'AAVEUSD' as asset FROM `development-302415.markets.AAVEUSD` UNION ALL SELECT time_stamp, price, volume, 'AAVEXBT' as asset FROM `development-302415.markets.AAVEXBT` UNION ALL SELECT time_stamp, price, volume, 'ADAETH' as asset FROM `development-302415.markets.ADAETH` UNION ALL SELECT time_stamp, price, volume, 'ADAEUR' as asset FROM `development-302415.markets.ADAEUR` UNION ALL SELECT time_stamp, price, volume, 'ADAUSD' as asset FROM `development-302415.markets.ADAUSD` UNION ALL SELECT time_stamp, price, volume, 'ADAUSDT' as asset FROM `development-302415.markets.ADAUSDT` UNION ALL SELECT time_stamp, price, volume, 'ADAXBT' as asset FROM `dev

In [15]:
%%bash 
gsutil ls gs://budfox-data-science-development/AggregateData/ | head 

gs://budfox-data-science-development/AggregateData/ADAUSDT_720.csv
gs://budfox-data-science-development/AggregateData/ALGOXBT_5.csv
gs://budfox-data-science-development/AggregateData/ANTXBT_60.csv
gs://budfox-data-science-development/AggregateData/ATOMETH_1440.csv
gs://budfox-data-science-development/AggregateData/ATOMETH_60.csv
gs://budfox-data-science-development/AggregateData/ATOMUSD_60.csv
gs://budfox-data-science-development/AggregateData/AUDJPY_1440.csv
gs://budfox-data-science-development/AggregateData/BALETH_5.csv
gs://budfox-data-science-development/AggregateData/BALUSD_15.csv
gs://budfox-data-science-development/AggregateData/BATUSD_15.csv


Exception ignored in: <_io.TextIOWrapper name='<stdout>' mode='w' encoding='UTF-8'>
BrokenPipeError: [Errno 32] Broken pipe


## Cleaning Up the tables that we no longer need now, after we merged them all into one, with variable `asset` 

In [35]:
for uri, table in zip(uris, assets):    
    table_id= "development-302415.markets." + table 
    client.delete_table(table_id, not_found_ok=True)  # Make an API request.
    print("Deleted table '{}'.".format(table_id))

Deleted table 'development-302415.markets.AAVEETH'.
Deleted table 'development-302415.markets.AAVEEUR'.
Deleted table 'development-302415.markets.AAVEGBP'.
Deleted table 'development-302415.markets.AAVEUSD'.
Deleted table 'development-302415.markets.AAVEXBT'.
Deleted table 'development-302415.markets.ADAETH'.
Deleted table 'development-302415.markets.ADAEUR'.
Deleted table 'development-302415.markets.ADAUSD'.
Deleted table 'development-302415.markets.ADAUSDT'.
Deleted table 'development-302415.markets.ADAXBT'.
Deleted table 'development-302415.markets.ALGOETH'.
Deleted table 'development-302415.markets.ALGOEUR'.
Deleted table 'development-302415.markets.ALGOUSD'.
Deleted table 'development-302415.markets.ALGOXBT'.
Deleted table 'development-302415.markets.ANTETH'.
Deleted table 'development-302415.markets.ANTEUR'.
Deleted table 'development-302415.markets.ANTUSD'.
Deleted table 'development-302415.markets.ANTXBT'.
Deleted table 'development-302415.markets.ATOMETH'.
Deleted table 'devel

## Preparing the Aggregate Tables 

In [53]:
blobs = storage_client.list_blobs(bucket_name)
all_aggregate_assets = [blob.name.split("/")[-1].split(".")[0] for blob in blobs if blob.name.split("/")[-2].split(".")[0]=='AggregateData']

In [54]:
print(all_aggregate_assets)

['AAVEETH_1', 'AAVEETH_1440', 'AAVEETH_15', 'AAVEETH_5', 'AAVEETH_60', 'AAVEETH_720', 'AAVEEUR_1', 'AAVEEUR_1440', 'AAVEEUR_15', 'AAVEEUR_5', 'AAVEEUR_60', 'AAVEEUR_720', 'AAVEGBP_1', 'AAVEGBP_1440', 'AAVEGBP_15', 'AAVEGBP_5', 'AAVEGBP_60', 'AAVEGBP_720', 'AAVEUSD_1', 'AAVEUSD_1440', 'AAVEUSD_15', 'AAVEUSD_5', 'AAVEUSD_60', 'AAVEUSD_720', 'AAVEXBT_1', 'AAVEXBT_1440', 'AAVEXBT_15', 'AAVEXBT_5', 'AAVEXBT_60', 'AAVEXBT_720', 'ADAETH_1', 'ADAETH_1440', 'ADAETH_15', 'ADAETH_5', 'ADAETH_60', 'ADAETH_720', 'ADAEUR_1', 'ADAEUR_1440', 'ADAEUR_15', 'ADAEUR_5', 'ADAEUR_60', 'ADAEUR_720', 'ADAUSDT_1', 'ADAUSDT_1440', 'ADAUSDT_15', 'ADAUSDT_5', 'ADAUSDT_60', 'ADAUSDT_720', 'ADAUSD_1', 'ADAUSD_1440', 'ADAUSD_15', 'ADAUSD_5', 'ADAUSD_60', 'ADAUSD_720', 'ADAXBT_1', 'ADAXBT_1440', 'ADAXBT_15', 'ADAXBT_5', 'ADAXBT_60', 'ADAXBT_720', 'ALGOETH_1', 'ALGOETH_1440', 'ALGOETH_15', 'ALGOETH_5', 'ALGOETH_60', 'ALGOETH_720', 'ALGOEUR_1', 'ALGOEUR_1440', 'ALGOEUR_15', 'ALGOEUR_5', 'ALGOEUR_60', 'ALGOEUR_720', 'AL

In [55]:
daily=[asset for asset in all_aggregate_assets if '720' in asset]

In [56]:
daily

['AAVEETH_720',
 'AAVEEUR_720',
 'AAVEGBP_720',
 'AAVEUSD_720',
 'AAVEXBT_720',
 'ADAETH_720',
 'ADAEUR_720',
 'ADAUSDT_720',
 'ADAUSD_720',
 'ADAXBT_720',
 'ALGOETH_720',
 'ALGOEUR_720',
 'ALGOUSD_720',
 'ALGOXBT_720',
 'ANTETH_720',
 'ANTEUR_720',
 'ANTUSD_720',
 'ANTXBT_720',
 'ATOMETH_720',
 'ATOMEUR_720',
 'ATOMUSD_720',
 'ATOMXBT_720',
 'AUDJPY_720',
 'AUDUSD_720',
 'ETHCHF_720',
 'LINKUSDT_720',
 'QTUMUSD_720',
 'UNIUSD_720',
 'YFIXBT_720',
 'BALETH_720',
 'BALEUR_720',
 'BALUSD_720',
 'BALXBT_720',
 'BATETH_720',
 'BATEUR_720',
 'BATUSD_720',
 'BATXBT_720',
 'BCHAUD_720',
 'BCHETH_720',
 'BCHEUR_720',
 'BCHGBP_720',
 'BCHJPY_720',
 'BCHUSDT_720',
 'BCHUSD_720',
 'BCHXBT_720',
 'COMPETH_720',
 'COMPEUR_720',
 'COMPUSD_720',
 'COMPXBT_720',
 'CRVETH_720',
 'CRVEUR_720',
 'CRVUSD_720',
 'CRVXBT_720',
 'DAIEUR_720',
 'DAIUSDT_720',
 'DAIUSD_720',
 'DASHEUR_720',
 'DASHUSD_720',
 'DASHXBT_720',
 'DOTETH_720',
 'DOTEUR_720',
 'DOTUSDT_720',
 'DOTUSD_720',
 'DOTXBT_720',
 'EOSETH_720'

In [57]:
daily_uris = ["gs://budfox-data-science-development/AggregateData/" + asset + ".csv" for asset in daily]
for i, uri in enumerate(daily_uris):
    if i > 10:
        break
    else:
        print(i, uri)

0 gs://budfox-data-science-development/AggregateData/AAVEETH_720.csv
1 gs://budfox-data-science-development/AggregateData/AAVEEUR_720.csv
2 gs://budfox-data-science-development/AggregateData/AAVEGBP_720.csv
3 gs://budfox-data-science-development/AggregateData/AAVEUSD_720.csv
4 gs://budfox-data-science-development/AggregateData/AAVEXBT_720.csv
5 gs://budfox-data-science-development/AggregateData/ADAETH_720.csv
6 gs://budfox-data-science-development/AggregateData/ADAEUR_720.csv
7 gs://budfox-data-science-development/AggregateData/ADAUSDT_720.csv
8 gs://budfox-data-science-development/AggregateData/ADAUSD_720.csv
9 gs://budfox-data-science-development/AggregateData/ADAXBT_720.csv
10 gs://budfox-data-science-development/AggregateData/ALGOETH_720.csv


In [58]:
# TODO(developer): Set table_id to the ID of the table to create.
# table_id = "your-project.your_dataset.your_table_name"

job_config = bigquery.LoadJobConfig(
    schema=[
        bigquery.SchemaField("time_stamp", "TIMESTAMP"),
        bigquery.SchemaField("Open", "FLOAT"),
        bigquery.SchemaField("High", "FLOAT"),
        bigquery.SchemaField("Low", "FLOAT"),
        bigquery.SchemaField("Close", "FLOAT"),
        bigquery.SchemaField("Volume", "FLOAT"),
        bigquery.SchemaField("Trades", "INTEGER")
        #bigquery.SchemaField("asset", "STRING")
    ],
    skip_leading_rows=0,
    # The source format defaults to CSV, so the line below is optional.
    source_format=bigquery.SourceFormat.CSV,
)
for uri, table in zip(daily_uris, daily):
        
    table_id= "development-302415.daily_aggregates." + table[:-4]
    
    load_job = client.load_table_from_uri(
        uri, table_id, job_config=job_config
    )  # Make an API request.

    load_job.result()  # Waits for the job to complete.

    destination_table = client.get_table(table_id)  # Make an API request.
    print("Loaded {} rows.".format(destination_table.num_rows))

Loaded 33 rows.
Loaded 33 rows.
Loaded 31 rows.
Loaded 33 rows.
Loaded 33 rows.
Loaded 1651 rows.
Loaded 1651 rows.
Loaded 310 rows.
Loaded 1651 rows.
Loaded 1651 rows.
Loaded 689 rows.
Loaded 689 rows.
Loaded 689 rows.
Loaded 1380 rows.
Loaded 73 rows.
Loaded 75 rows.
Loaded 75 rows.
Loaded 75 rows.
Loaded 1233 rows.
Loaded 1239 rows.
Loaded 1239 rows.
Loaded 1239 rows.
Loaded 181 rows.
Loaded 393 rows.
Loaded 2337 rows.
Loaded 462 rows.
Loaded 4950 rows.
Loaded 465 rows.
Loaded 465 rows.
Loaded 204 rows.
Loaded 211 rows.
Loaded 211 rows.
Loaded 422 rows.
Loaded 978 rows.
Loaded 995 rows.
Loaded 1990 rows.
Loaded 995 rows.
Loaded 338 rows.
Loaded 489 rows.
Loaded 2494 rows.
Loaded 428 rows.
Loaded 74 rows.
Loaded 482 rows.
Loaded 2496 rows.
Loaded 2495 rows.
Loaded 339 rows.
Loaded 339 rows.
Loaded 678 rows.
Loaded 339 rows.
Loaded 211 rows.
Loaded 211 rows.
Loaded 211 rows.
Loaded 211 rows.
Loaded 927 rows.
Loaded 921 rows.
Loaded 927 rows.
Loaded 2716 rows.
Loaded 2716 rows.
Loaded 

In [65]:
tables = client.list_tables("development-302415.daily_aggregates")

In [66]:
len([table for table in tables])

251

In [71]:
queries = ["SELECT time_stamp, Open, High, Low, Close, Volume, Trades, '" + asset + "' as asset FROM `development-302415.daily_aggregates." + asset + "`" for asset in assets]
for q in queries[:3]:
    print(q)

SELECT time_stamp, Open, High, Low, Close, Volume, Trades, 'AAVEETH' as asset FROM `development-302415.daily_aggregates.AAVEETH`
SELECT time_stamp, Open, High, Low, Close, Volume, Trades, 'AAVEEUR' as asset FROM `development-302415.daily_aggregates.AAVEEUR`
SELECT time_stamp, Open, High, Low, Close, Volume, Trades, 'AAVEGBP' as asset FROM `development-302415.daily_aggregates.AAVEGBP`


In [72]:
%%bigquery results
SELECT time_stamp, Open, High, Low, Close, Volume, Trades, 'AAVEEUR' as asset FROM `development-302415.daily_aggregates.AAVEEUR`


In [73]:
results.head()

,time_stamp,Open,High,Low,Close,Volume,Trades,asset
0,2020-12-30 12:00:00+00:00,65.15,68.80,63.14,66.30,4303.156851,260,AAVEEUR
1,2020-12-16 00:00:00+00:00,73.29,74.89,66.98,67.63,480.202285,275,AAVEEUR
2,2020-12-26 00:00:00+00:00,65.06,65.06,61.79,63.03,145.164305,28,AAVEEUR
3,2020-12-25 00:00:00+00:00,66.45,66.85,63.39,66.85,248.425558,50,AAVEEUR
4,2020-12-23 12:00:00+00:00,63.25,69.80,52.28,61.82,1576.489909,315,AAVEEUR


In [74]:
giant_query = " UNION ALL ".join(queries)

In [75]:
print(giant_query)

SELECT time_stamp, Open, High, Low, Close, Volume, Trades, 'AAVEETH' as asset FROM `development-302415.daily_aggregates.AAVEETH` UNION ALL SELECT time_stamp, Open, High, Low, Close, Volume, Trades, 'AAVEEUR' as asset FROM `development-302415.daily_aggregates.AAVEEUR` UNION ALL SELECT time_stamp, Open, High, Low, Close, Volume, Trades, 'AAVEGBP' as asset FROM `development-302415.daily_aggregates.AAVEGBP` UNION ALL SELECT time_stamp, Open, High, Low, Close, Volume, Trades, 'AAVEUSD' as asset FROM `development-302415.daily_aggregates.AAVEUSD` UNION ALL SELECT time_stamp, Open, High, Low, Close, Volume, Trades, 'AAVEXBT' as asset FROM `development-302415.daily_aggregates.AAVEXBT` UNION ALL SELECT time_stamp, Open, High, Low, Close, Volume, Trades, 'ADAETH' as asset FROM `development-302415.daily_aggregates.ADAETH` UNION ALL SELECT time_stamp, Open, High, Low, Close, Volume, Trades, 'ADAEUR' as asset FROM `development-302415.daily_aggregates.ADAEUR` UNION ALL SELECT time_stamp, Open, High, 

In [76]:
for uri, table in zip(daily_uris, assets):    
    table_id= "development-302415.daily_aggregates." + table 
    client.delete_table(table_id, not_found_ok=True)  # Make an API request.
    print("Deleted table '{}'.".format(table_id))

Deleted table 'development-302415.daily_aggregates.AAVEETH'.
Deleted table 'development-302415.daily_aggregates.AAVEEUR'.
Deleted table 'development-302415.daily_aggregates.AAVEGBP'.
Deleted table 'development-302415.daily_aggregates.AAVEUSD'.
Deleted table 'development-302415.daily_aggregates.AAVEXBT'.
Deleted table 'development-302415.daily_aggregates.ADAETH'.
Deleted table 'development-302415.daily_aggregates.ADAEUR'.
Deleted table 'development-302415.daily_aggregates.ADAUSD'.
Deleted table 'development-302415.daily_aggregates.ADAUSDT'.
Deleted table 'development-302415.daily_aggregates.ADAXBT'.
Deleted table 'development-302415.daily_aggregates.ALGOETH'.
Deleted table 'development-302415.daily_aggregates.ALGOEUR'.
Deleted table 'development-302415.daily_aggregates.ALGOUSD'.
Deleted table 'development-302415.daily_aggregates.ALGOXBT'.
Deleted table 'development-302415.daily_aggregates.ANTETH'.
Deleted table 'development-302415.daily_aggregates.ANTEUR'.
Deleted table 'development-302